In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [ ]:
# Set the paths to the dataset folders
train_dir = 'train/dataset'
test_dir = 'test/dataset'

In [ ]:
# Define image data generators for train and test sets
train_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Rescale pixel values between 0 and 1
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
# Set the image size and batch size
image_size = (150, 150)
batch_size = 16

In [ ]:
# Generate train and test sets from the directory structure
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Load pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Create the model
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
num_epochs = 50
history = model.fit(train_generator, epochs=num_epochs, validation_data=test_generator)

# Save the trained model
model.save('keras_model.h5')